In [42]:
import os
import random
import pandas as pd
import numpy as np
from pprint import pprint
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix
from scipy.stats import spearmanr
import itertools
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
import shap 
import seaborn as sns
sns.set_style('darkgrid')

# Define file paths

In [43]:
MEASURES_FILE = "MdRQA_measures.csv"
SHUFF_MEASURES_FILE = "shuff_MdRQA_measures.csv"
LABELS_FILE = "team_block_outcomes.csv"
TASK_SCORE_RESULTS = "results/task_score/"
SUBJ_OUTCOME_RESULTS = "results/subjective_outcome/"
VALENCE_RESULTS = "results/valence/"

# Load data

In [44]:
dfMeasures = pd.read_csv(MEASURES_FILE)
dfShuffMeasures = pd.read_csv(SHUFF_MEASURES_FILE)
dfLabels = pd.read_csv(LABELS_FILE)

print("%s shape: %s" % (MEASURES_FILE, dfMeasures.shape))
print("%s shape: %s" % (SHUFF_MEASURES_FILE, dfShuffMeasures.shape))
print("%s shape: %s" % (LABELS_FILE, dfLabels.shape))


MdRQA_measures.csv shape: (271, 11)
shuff_MdRQA_measures.csv shape: (271, 11)
team_block_outcomes.csv shape: (274, 7)


In [45]:
features = ['REC', 'DET', 'ADL', 'MDL', 'DENTR', 'LAM', 'AVL', 'MVL', 'VENTR']


def get_group_data(dfData, GROUPID_list, features_list, label_names):
    dfGroupsData = pd.DataFrame()    
    for GROUPID in GROUPID_list:
        dfGroupsData = pd.concat([dfGroupsData, dfData.loc[dfData['GROUPID'] == GROUPID, :]], ignore_index=True)
        
    data = dfGroupsData.loc[:, features_list]  
    labels = dfGroupsData.loc[:, label_names]

    return [data, labels, dfGroupsData]



# Experiment Set Up

In [52]:
# Get model type
shuffled = False
chance = False
num_iters = 25
num_folds = 10
model = "RFR"

if shuffled:
    dfData = pd.merge(dfShuffMeasures, dfLabels, on=['GROUPID', 'block'], how='inner')
    # Drop rows with NaN for ADL or AVL
    dfData = dfData.dropna()
elif chance:
    dfData = pd.merge(dfMeasures, dfLabels, on=['GROUPID', 'block'], how='inner')
    # Shuffle labels
    label_cols = ['CPS_and_ITN_mean', 'Valence', 'num_gold', 'num_silver', 'task_score']
    dfData.loc[:, label_cols] = shuffle(dfData.loc[:, label_cols], random_state=12).reset_index(drop=True)
else:
    dfData = pd.merge(dfMeasures, dfLabels, on=['GROUPID', 'block'], how='inner')


print("dfData shape: ", dfData.shape)
display(dfData)


dfData shape:  (271, 16)


,GROUPID,block,REC,DET,ADL,MDL,DENTR,LAM,AVL,MVL,VENTR,CPS_and_ITN_mean,Valence,num_gold,num_silver,task_score
0,1010,ExpBlock1,0.111890,46.765002,2.580148,12,8.031479,63.476117,3.202765,8,8.276899,-0.413762,3.666667,1.0,2.0,4.0
1,1010,ExpBlock2,0.221818,51.941830,2.801763,13,8.693110,71.437617,3.567544,14,8.801583,0.662988,5.000000,3.0,2.0,8.0
2,10100,ExpBlock1,0.469005,68.603876,3.236574,27,9.502080,83.376781,4.480822,28,9.315484,-0.780143,2.666667,0.0,0.0,0.0
3,10100,ExpBlock2,0.328632,63.319890,3.373809,34,8.995863,78.390610,4.336822,35,8.899755,-0.290702,3.000000,0.0,1.0,1.0
4,10102,ExpBlock1,0.275806,40.010880,2.781595,16,8.650819,60.603101,3.351069,15,8.895810,0.425265,3.000000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,2070,ExpBlock2,0.128570,35.135661,2.392715,9,7.967704,59.342180,2.886733,9,8.447403,0.328772,3.666667,1.0,1.0,3.0
267,2070,Warmup,0.411115,49.651775,2.839304,42,9.178902,70.710415,3.726037,43,9.143787,-0.627717,3.666667,5.0,3.0,13.0
268,2071,ExpBlock1,0.291773,45.826191,2.544979,11,8.971522,68.343464,3.244426,12,9.145826,-0.568987,3.000000,0.0,0.0,0.0
269,2071,ExpBlock2,0.188310,46.079278,2.572276,16,8.522221,66.710627,3.169010,17,8.746838,-0.255739,3.666667,0.0,0.0,0.0


## Prep for team-level cross validation

In [47]:
# Define fold names
train_folds = []
test_folds = []
set_type = "test"
for j in range(1,num_folds+1): 
    col_name = "Fold" + str(j) + "_" + set_type
    test_folds.append(col_name)
    set_type = "train"  
    col_name = "Fold" + str(j) + "_" + set_type
    train_folds.append(col_name)
    set_type = "test"

folds_dict_list = []

# Split teams into 5 groups
teams = pd.unique(dfMeasures.GROUPID)

# For every iteration
for i in range(1,num_iters+1):
    print("Iteration: ", i)
    teams = shuffle(teams, random_state=i)
    groups = np.array_split(teams, num_folds)
    
    # Define groups for each fold
    fold_groups = {}
    for j, (train_fold, test_fold) in enumerate(zip(train_folds, test_folds)):
        # make the current group the test group
        fold_groups[test_fold] = groups[j]
        # make all other groups the train group
        train_group = groups[:j] + groups[j+1:]
        train_group = [team for group in train_group for team in group]
        fold_groups[train_fold] = train_group
        
    ## Confirm that for each fold, there is no team overlap bewteen train and test set
    for j in range(1,num_folds+1):
        assert set(fold_groups['Fold'+str(j)+'_test']).isdisjoint(set(fold_groups['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
    print("* No team overlap *")

      
    # Add fold groups to dictionary
    folds_dict_list.append(fold_groups)
    

# Informational
print("\nNumber of iterations: ", len(folds_dict_list))

print("\nIterating through folds_dict_list to check for overlap...")
for i,dicti in enumerate(folds_dict_list):
    for j in range(1,num_folds+1):
        assert set(dicti['Fold'+str(j)+'_test']).isdisjoint(set(dicti['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
print("* No team overlap *")  



Iteration:  1
* No team overlap *
Iteration:  2
* No team overlap *
Iteration:  3
* No team overlap *
Iteration:  4
* No team overlap *
Iteration:  5
* No team overlap *
Iteration:  6
* No team overlap *
Iteration:  7
* No team overlap *
Iteration:  8
* No team overlap *
Iteration:  9
* No team overlap *
Iteration:  10
* No team overlap *
Iteration:  11
* No team overlap *
Iteration:  12
* No team overlap *
Iteration:  13
* No team overlap *
Iteration:  14
* No team overlap *
Iteration:  15
* No team overlap *
Iteration:  16
* No team overlap *
Iteration:  17
* No team overlap *
Iteration:  18
* No team overlap *
Iteration:  19
* No team overlap *
Iteration:  20
* No team overlap *
Iteration:  21
* No team overlap *
Iteration:  22
* No team overlap *
Iteration:  23
* No team overlap *
Iteration:  24
* No team overlap *
Iteration:  25
* No team overlap *

Number of iterations:  25

Iterating through folds_dict_list to check for overlap...
* No team overlap *


In [7]:
# # Define fold names
# train_folds = []
# test_folds = []
# set_type = "test"
# for j in range(1,num_folds+1): 
#     col_name = "Fold" + str(j) + "_" + set_type
#     test_folds.append(col_name)
#     set_type = "train"  
#     col_name = "Fold" + str(j) + "_" + set_type
#     train_folds.append(col_name)
#     set_type = "test"

# # print("Train fold names: ", train_folds)
# # print("Test fold names: ",test_folds)


# folds_dict_list = []

# # Split teams into 5 groups
# teams = pd.unique(dfMeasures.GROUPID)

# # For every iteration
# for i in range(1,num_iters+1):
#     print("\n\n=============Iteration: ", i)
#     random.Random(i).shuffle(teams)

# #     teams = shuffle(teams, random_state=i)

#     groups = np.array_split(teams, num_folds)
#     print("\ngroups: ")
#     for k,grp in enumerate(groups):
#         print(k, grp)
    
#     # Define groups for each fold
#     fold_groups = {}
#     for j, (train_fold, test_fold) in enumerate(zip(train_folds, test_folds)):
#         # make the current group the test group
#         fold_groups[test_fold] = groups[j]
#         # make all other groups the train group
#         train_group = groups[:j] + groups[j+1:]
#         train_group = [team for group in train_group for team in group]
#         fold_groups[train_fold] = train_group
        
#     ## Confirm that for each fold, there is no team overlap bewteen train and test set
#     for j in range(1,num_folds+1):
#         assert set(fold_groups['Fold'+str(j)+'_test']).isdisjoint(set(fold_groups['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
#     print("* No team overlap *")
    
#     print("\n!!!!! FOLD GROUPS BEFORE IT GOES BAD")
#     for kei in fold_groups:
#         print("\n", kei)
#         print(fold_groups[kei])

        
#     if i == 2:
#         print("\n BEFORE APPEND folds_dict_list[i-2] index = ", i-2)
#         folds_dict = folds_dict_list[i-2]
#         for key in folds_dict:
#             print("\n", key)
#             print(folds_dict[key])
      
#     # Add fold groups to dictionary
#     folds_dict_list.append(fold_groups)
    
#     if i == 2:
#         print("\n AFTER APPEND folds_dict_list[i-2] index = ", i-2)
#         folds_dict = folds_dict_list[i-2]
#         for key in folds_dict:
#             print("\n", key)
#             print(folds_dict[key])   

# # Informational
# # print("\nNumber of iterations: ", len(folds_dict_list))

# print("\n*~*~*~*~*~* iterate through folds_dict_list: ")
# for i,dicti in enumerate(folds_dict_list):
#     print("\n list index: ", i)
#     for j in range(1,num_folds+1):
#         assert set(dicti['Fold'+str(j)+'_test']).isdisjoint(set(dicti['Fold'+str(j)+'_train'])), "There is overlap in train and test set " + str(j)
    
    


# Regression

## Predict Task Score

In [41]:
# Resources
## https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

# Store metrics for all iterations
maes = []    # MAE (mean absolute errors)
mses = []    # MSE (mean squared errors)
rmses = []   # RMSE (root mean squared errors)
corrs = []   # spearman correlations
ps = []      # spearman correlation p-values

all_y_test_task_score = [[] for i in range(num_iters)]
predictions_task_score = [[] for i in range(num_iters)]
predict_proba_task_score = [[] for i in range(num_iters)]

# For storing shap values across all folds
task_score_shap_values_0 = None
task_score_shap_values_1 = None
task_score_full_X_test = pd.DataFrame()

#-----------------------------------------------#
#      5-fold team level cross-validation       #
#-----------------------------------------------#

# For each iteration 
for i in range(num_iters):
    print("Iteration: ", i+1)
    
    # Lists for cumulative test set and predictions for iteration
    dfFullTest = pd.DataFrame()
    all_y_test = []
    predictions = []
    
    # Create model for task_score prediction
    if model == "RFR":
        model_task_score = RandomForestRegressor(n_estimators=100, random_state=1, max_features='sqrt') 

    elif model == "SVR":
        model_task_score = SVR()
    
    
    # Get fold groups
    fold_groups = folds_dict_list[i]
    
    # For each fold
    for j, (test_fold, train_fold) in enumerate(zip(test_folds, train_folds)):
#         print("\tFold: ", j+1)
        # Get data for teams in test set
        test_data_list = get_group_data(dfData, fold_groups[test_fold], features, 'task_score')
        X_test = test_data_list[0]
        y_test = test_data_list[1]
        all_y_test.extend(y_test.tolist())

        dfFullTest = pd.concat([dfFullTest, test_data_list[2]], ignore_index=True)      
        
        # Get data for teams in train set
        train_data_list = get_group_data(dfData, fold_groups[train_fold], features, 'task_score')
        X_train = train_data_list[0]
        y_train = train_data_list[1]

        # Train model
        model_task_score.fit(X_train, y_train)

        # Test model
        y_pred = model_task_score.predict(X_test)
        predictions.extend(y_pred.tolist())
        
#         ## Get SHAP values
#         explainer = shap.TreeExplainer(model_task_scorerfc_task_score)
#         shap_values = explainer.shap_values(X_test)
        
#         if j==0:
#             task_score_shap_values_0 = shap_values[0]
#             task_score_shap_values_1 = shap_values[1]
#         else:
#             task_score_shap_values_0 = np.vstack([task_score_shap_values_0, shap_values[0]])
#             task_score_shap_values_1 = np.vstack([task_score_shap_values_1, shap_values[1]])
#         task_score_full_X_test = pd.concat([task_score_full_X_test, X_test], ignore_index=True)
#         ## End of get SHAP values

# ----- END OF FOLDS
    
    all_y_test = np.array(all_y_test)
    all_y_test_task_score[i] = all_y_test

    predictions = np.array(predictions)
    predictions_task_score[i] = predictions

    # Calculate the absolute errors (MAE) of the iteration
    mae = metrics.mean_absolute_error(all_y_test, predictions)
    mse = metrics.mean_squared_error(all_y_test, predictions)
    rmse = np.sqrt(metrics.mean_squared_error(all_y_test, predictions))
    corr, p = spearmanr(all_y_test, predictions)
    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
    corrs.append(corr)
    ps.append(p)

     # Save actual labels and predictions for iteration
    dfTruevPred = dfFullTest.loc[:, ['GROUPID', 'block', 'task_score']]
    dfTruevPred['prediction'] = predictions
    dfTruevPred.sort_values(['GROUPID', 'block', 'task_score'], ignore_index=True, inplace=True)

    dfTruevPred['error'] = abs(dfTruevPred['prediction'] - dfTruevPred['task_score'])
#     print("MAE from df: ", round(np.mean(dfTruevPred['error']), 2))
#     # FOR PLOTTING ACTUAL vs. PREDICTED   
#     dfTruevPred.plot(y=['task_score', 'prediction'], title='Actual vs. Predicted Task Score', \
#                      style=['b-', 'ro'], figsize=(20, 5))
#     # END PLOTTING
        
    
    if shuffled:
        dfTruevPred.to_csv(TASK_SCORE_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_TaskScore_True_vs_Pred_SHUFF_" + str(i+1) + ".csv", index=False)
    elif chance:
        dfTruevPred.to_csv(TASK_SCORE_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_TaskScore_True_vs_Pred_CHANCE_" + str(i+1) + ".csv", index=False)
    else:
        dfTruevPred.to_csv(TASK_SCORE_RESULTS + "RAW/" + model + "/" + model + "_TaskScore_True_vs_Pred_" + str(i+1) + ".csv", index=False)
  

    
#     if model == "RFR":    
#         ### Compute Feature Importances for last fold iteration ###
#         # Impurity-based importances
#         importances = model_task_score.feature_importances_
#         std = np.std([tree.feature_importances_ for tree in model_task_score.estimators_], axis=0)
#         imps = pd.Series(importances, index=features)
#         stds = pd.Series(std, index=features)
#         ## Plot feature importances
#         fig1, ax1 = plt.subplots()
#         imps.plot.bar(yerr=std, ax=ax1)
#         ax1.set_title("Task Score Feature importances using MDI")
#         ax1.set_ylabel("Mean decrease in impurity")
#         plt.show()
#         ## END plotting feature importances
    
    
# ----- END OF ITERATIONS

print("\n =========== ALL ITERATIONS RESULTS SUMMARY ===========")
dfMetrics = pd.DataFrame({'iteration': [i for i in range(1,num_iters+1)], \
                          'mae': maes, 'mse': mses, 'rmse': rmses, 'corrs': corrs, 'ps': ps})

display(dfMetrics)

if shuffled:
    dfMetrics.to_csv(TASK_SCORE_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_TaskScore_Metrics_SHUFF.csv", index=False)
elif chance:
    dfMetrics.to_csv(TASK_SCORE_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_TaskScore_Metrics_CHANCE.csv", index=False)
else:
    dfMetrics.to_csv(TASK_SCORE_RESULTS + "RAW/" + model + "/" + model + "_TaskScore_Metrics.csv", index=False)


print("Average over all iterations:")
print("%6s %.2f" % ("MAE:", np.mean(dfMetrics['mae'])))
print("%6s %.2f" % ("MSE:", np.mean(dfMetrics['mse'])))
print("%6s %.2f" % ("RMSE:", np.mean(dfMetrics['rmse'])))
print("%6s %.2f" % ("Corr:", np.mean(dfMetrics['corrs'])))
print("%6s %.7f" % ("p-val:", np.mean(dfMetrics['ps'])))

    

Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25

 =========== ALL ITERATIONS RESULTS SUMMARY ===========


,iteration,mae,mse,rmse,corrs,ps
0,1,2.578229,11.132506,3.336541,0.325886,4.006128e-08
1,2,2.614317,11.453538,3.384308,0.312814,1.453738e-07
2,3,2.598598,11.139976,3.337660,0.308911,2.111163e-07
3,4,2.575904,11.104554,3.332350,0.346134,4.808016e-09
4,5,2.639483,11.736855,3.425909,0.334542,1.649056e-08
5,6,2.688819,11.907335,3.450701,0.297437,6.128237e-07
6,7,2.655830,11.374772,3.372651,0.331465,2.268048e-08
7,8,2.682952,12.048705,3.471124,0.284311,1.962453e-06
8,9,2.597306,11.256963,3.355140,0.333423,1.852329e-08
9,10,2.586974,11.304858,3.362270,0.334956,1.579365e-08


Average over all iterations:
  MAE: 2.62
  MSE: 11.46
 RMSE: 3.38
 Corr: 0.32
p-val: 0.0000006


# Predict Subjective Outcome (combined CPS and ITN score)

In [48]:
# Resources
## https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

# Store metrics for all iterations
# aurocs = []
maes = []    # MAE (mean absolute errors)
mses = []    # MSE (mean squared errors)
rmses = []   # RMSE (root mean squared errors)
corrs = []   # spearman correlations
ps = []      # spearman correlation p-values

all_y_test_subj_out = [[] for i in range(num_iters)]
predictions_subj_out = [[] for i in range(num_iters)]

# For storing shap values across all folds
subj_out_shap_values_0 = None
subj_out_shap_values_1 = None
subj_out_full_X_test = pd.DataFrame()

#-----------------------------------------------#
#      5-fold team level cross-validation       #
#-----------------------------------------------#

# For each iteration 
for i in range(num_iters):
    print("Iteration: ", i+1)
    
    # Lists for cumulative test set and predictions for iteration
    dfFullTest = pd.DataFrame()
    all_y_test = []
    predictions = []
    
    # Create model for task_score prediction
    if model == "RFR":
        model_subj_out = RandomForestRegressor(n_estimators=100, random_state=1, \
                                           max_features='sqrt') 
    elif model == "SVR":
        model_subj_out = SVR()
    
    # Get fold groups
    fold_groups = folds_dict_list[i]
    
    # For each fold
    for j, (test_fold, train_fold) in enumerate(zip(test_folds, train_folds)):
#         print("\tFold: ", j+1)
        # Get data for teams in test set
        test_data_list = get_group_data(dfData, fold_groups[test_fold], features, 'CPS_and_ITN_mean')
        X_test = test_data_list[0]
        y_test = test_data_list[1]
        all_y_test.extend(y_test.tolist())
        dfFullTest = pd.concat([dfFullTest, test_data_list[2]], ignore_index=True)
        
        # Get data for teams in train set
        train_data_list = get_group_data(dfData, fold_groups[train_fold], features, 'CPS_and_ITN_mean')
        X_train = train_data_list[0]
        y_train = train_data_list[1]
        

        # Train model
        model_subj_out.fit(X_train, y_train)

        # Test model
        y_pred = model_subj_out.predict(X_test)
        predictions.extend(y_pred.tolist())
        
#         ## Get SHAP values
#         explainer = shap.TreeExplainer(model_subj_outrfc_subj_out)
#         shap_values = explainer.shap_values(X_test)
        
#         if j==0:
#             subj_out_shap_values_0 = shap_values[0]
#             subj_out_shap_values_1 = shap_values[1]
#         else:
#             subj_out_shap_values_0 = np.vstack([subj_out_shap_values_0, shap_values[0]])
#             subj_out_shap_values_1 = np.vstack([subj_out_shap_values_1, shap_values[1]])
#         subj_out_full_X_test = pd.concat([subj_out_full_X_test, X_test], ignore_index=True)
#         ## End of get SHAP values

# ----- END OF FOLDS

    # Calculate the absolute errors (MAE) of the iteration
    predictions = np.array(predictions)
    all_y_test = np.array(all_y_test)
    mae = metrics.mean_absolute_error(all_y_test, predictions)
    mse = metrics.mean_squared_error(all_y_test, predictions)
    rmse = np.sqrt(metrics.mean_squared_error(all_y_test, predictions))
    corr, p = spearmanr(all_y_test, predictions)
    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
    corrs.append(corr)
    ps.append(p)
    
    # Save iteration subjective outcome truth labels, and predictions for stats across all iterations
    all_y_test_subj_out[i] = all_y_test
    predictions_subj_out[i] = predictions

    # Save actual labels and predictions for iteration
    dfTruevPred = dfFullTest.loc[:, ['GROUPID', 'block', 'CPS_and_ITN_mean']]
    dfTruevPred['prediction'] = predictions
    dfTruevPred.sort_values(['GROUPID', 'block', 'CPS_and_ITN_mean'], ignore_index=True, inplace=True)
    dfTruevPred['error'] = abs(dfTruevPred['prediction'] - dfTruevPred['CPS_and_ITN_mean'])
#     print("MAE from df: ", round(np.mean(dfTruevPred['error']), 2))
    
    if shuffled:
        dfTruevPred.to_csv(SUBJ_OUTCOME_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_SubjOut_True_vs_Pred_SHUFF_" + str(i+1) + ".csv", index=False)
    elif chance:
        dfTruevPred.to_csv(SUBJ_OUTCOME_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_SubjOut_True_vs_Pred_CHANCE_" + str(i+1) + ".csv", index=False)
    else:
        dfTruevPred.to_csv(SUBJ_OUTCOME_RESULTS + "RAW/" + model + "/" + model + "_SubjOut_True_vs_Pred_" + str(i+1) + ".csv", index=False)
    
#     # FOR PLOTTING ACTUAL vs. PREDICTED    
#     dfTruevPred.plot(y=['CPS_and_ITN_mean', 'prediction'], title='Actual vs. Predicted Subjective Outcome', \
#                      style=['b-', 'ro'], figsize=(20, 5))
#     # END PLOTTING
    
#     if model == "RFR":
#         ### Compute Feature Importances for last fold iteration ###
#         # Impurity-based importances
#         importances = model_subj_out.feature_importances_
#         std = np.std([tree.feature_importances_ for tree in model_subj_out.estimators_], axis=0)
#         imps = pd.Series(importances, index=features)
#         stds = pd.Series(std, index=features)
#         ## Plot feature importances
#         fig1, ax1 = plt.subplots()
#         imps.plot.bar(yerr=std, ax=ax1)
#         ax1.set_title("CPS and ITN Feature importances using MDI")
#         ax1.set_ylabel("Mean decrease in impurity")
#         plt.show()
#         ## END plotting feature importances
    
    
# ----- END OF ITERATIONS

print("\n =========== ALL ITERATIONS RESULTS SUMMARY ===========")
dfMetrics = pd.DataFrame({'iteration': [i for i in range(1,num_iters+1)], \
                          'mae': maes, 'mse': mses, 'rmse': rmses, 'corrs': corrs, 'ps': ps})
display(dfMetrics)

if shuffled:
    dfMetrics.to_csv(SUBJ_OUTCOME_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_SubjOut_Metrics_SHUFF.csv", index=False)
elif chance:
    dfMetrics.to_csv(SUBJ_OUTCOME_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_SubjOut_Metrics_CHANCE.csv", index=False)
else:
    dfMetrics.to_csv(SUBJ_OUTCOME_RESULTS + "RAW/" + model + "/" + model + "_SubjOut_Metrics.csv", index=False)

print("Average over all iterations:")
print("%6s %.2f" % ("MAE:", np.mean(dfMetrics['mae'])))
print("%6s %.2f" % ("MSE:", np.mean(dfMetrics['mse'])))
print("%6s %.2f" % ("RMSE:", np.mean(dfMetrics['rmse'])))
print("%6s %.2f" % ("Corr:", np.mean(dfMetrics['corrs'])))
print("%6s %.7f" % ("p-val:", np.mean(dfMetrics['ps'])))



Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25

 =========== ALL ITERATIONS RESULTS SUMMARY ===========


,iteration,mae,mse,rmse,corrs,ps
0,1,0.448270,0.323950,0.569166,0.007989,0.895847
1,2,0.440088,0.315805,0.561966,0.061461,0.313430
2,3,0.450520,0.323464,0.568739,-0.003247,0.957569
3,4,0.443326,0.323420,0.568701,0.011029,0.856580
4,5,0.441004,0.319461,0.565209,0.004888,0.936162
5,6,0.434989,0.315003,0.561251,0.061055,0.316642
6,7,0.437171,0.313981,0.560340,0.021744,0.721585
7,8,0.446004,0.323601,0.568860,-0.010053,0.869155
8,9,0.447429,0.327646,0.572403,0.003018,0.960564
9,10,0.455344,0.333953,0.577887,-0.034057,0.576694


Average over all iterations:
  MAE: 0.44
  MSE: 0.32
 RMSE: 0.57
 Corr: 0.01
p-val: 0.7672213


# Predict Valence

In [31]:
# Resources
## https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

# Store metrics for all iterations
# aurocs = []
maes = []    # MAE (mean absolute errors)
mses = []    # MSE (mean squared errors)
rmses = []   # RMSE (root mean squared errors)
corrs = []   # spearman correlations
ps = []      # spearman correlation p-values

all_y_test_valence = [[] for i in range(num_iters)]
predictions_valence = [[] for i in range(num_iters)]

# For storing shap values across all folds
valence_shap_values_0 = None
valence_shap_values_1 = None
valence_full_X_test = pd.DataFrame()

#-----------------------------------------------#
#      5-fold team level cross-validation       #
#-----------------------------------------------#

# For each iteration 
for i in range(num_iters):
    print("Iteration: ", i+1)
    
    # Lists for cumulative test set and predictions for iteration
    dfFullTest = pd.DataFrame()
    all_y_test = []
    predictions = []
    
    # Create model for task_score prediction
    if model == "RFR":
        model_valence = RandomForestRegressor(n_estimators=100, random_state=1, \
                                           max_features='sqrt') 
    elif model == "SVR":
        model_valence = SVR()
    
    # Get fold groups
    fold_groups = folds_dict_list[i]
    
    # For each fold
    for j, (test_fold, train_fold) in enumerate(zip(test_folds, train_folds)):
#         print("\tFold: ", j+1)
        # Get data for teams in test set
        test_data_list = get_group_data(dfData, fold_groups[test_fold], features, 'Valence')
        X_test = test_data_list[0]
        y_test = test_data_list[1]
        all_y_test.extend(y_test.tolist())
        dfFullTest = pd.concat([dfFullTest, test_data_list[2]], ignore_index=True)
        
        # Get data for teams in train set
        train_data_list = get_group_data(dfData, fold_groups[train_fold], features, 'Valence')
        X_train = train_data_list[0]
        y_train = train_data_list[1]
        

        # Train model
        model_valence.fit(X_train, y_train)

        # Test model
        y_pred = model_valence.predict(X_test)
        predictions.extend(y_pred.tolist())
        
#         ## Get SHAP values
#         explainer = shap.TreeExplainer(model_valencerfc_valence)
#         shap_values = explainer.shap_values(X_test)
        
#         if j==0:
#             valence_shap_values_0 = shap_values[0]
#             valence_shap_values_1 = shap_values[1]
#         else:
#             valence_shap_values_0 = np.vstack([valence_shap_values_0, shap_values[0]])
#             valence_shap_values_1 = np.vstack([valence_shap_values_1, shap_values[1]])
#         valence_full_X_test = pd.concat([valence_full_X_test, X_test], ignore_index=True)
#         ## End of get SHAP values

# ----- END OF FOLDS

    # Calculate the absolute errors (MAE) of the iteration
    predictions = np.array(predictions)
    all_y_test = np.array(all_y_test)
    mae = metrics.mean_absolute_error(all_y_test, predictions)
    mse = metrics.mean_squared_error(all_y_test, predictions)
    rmse = np.sqrt(metrics.mean_squared_error(all_y_test, predictions))
    corr, p = spearmanr(all_y_test, predictions)
    maes.append(mae)
    mses.append(mse)
    rmses.append(rmse)
    corrs.append(corr)
    ps.append(p)
    
    # Save iteration valence truth labels, and predictions for stats across all iterations
    all_y_test_subj_out[i] = all_y_test
    predictions_subj_out[i] = predictions

    # Save actual labels and predictions for iteration
    dfTruevPred = dfFullTest.loc[:, ['GROUPID', 'block', 'Valence']]
    dfTruevPred['prediction'] = predictions
    dfTruevPred.sort_values(['GROUPID', 'block', 'Valence'], ignore_index=True, inplace=True)
    dfTruevPred['error'] = abs(dfTruevPred['prediction'] - dfTruevPred['Valence'])
#     print("MAE from df: ", round(np.mean(dfTruevPred['error']), 2))
    
    if shuffled:
        dfTruevPred.to_csv(VALENCE_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_Valence_True_vs_Pred_SHUFF_" + str(i+1) + ".csv", index=False)
    elif chance:
        dfTruevPred.to_csv(VALENCE_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_Valence_True_vs_Pred_CHANCE_" + str(i+1) + ".csv", index=False)
    else:
        dfTruevPred.to_csv(VALENCE_RESULTS + "RAW/" + model + "/" + model + "_Valence_True_vs_Pred_" + str(i+1) + ".csv", index=False)
    
#     # FOR PLOTTING ACTUAL vs. PREDICTED   
#     dfTruevPred.plot(y=['Valence', 'prediction'], title='Actual vs. Predicted Valence', \
#                      style=['b-', 'ro'], figsize=(20, 5))
#     # END PLOTTING
    
#     if model == "RFR":
#         ### Compute Feature Importances for last fold iteration ###
#         # Impurity-based importances
#         importances = model_valence.feature_importances_
#         std = np.std([tree.feature_importances_ for tree in model_valence.estimators_], axis=0)
#         imps = pd.Series(importances, index=features)
#         stds = pd.Series(std, index=features)
#         ## Plot feature importances
#         fig1, ax1 = plt.subplots()
#         imps.plot.bar(yerr=std, ax=ax1)
#         ax1.set_title("Valence Feature importances using MDI")
#         ax1.set_ylabel("Mean decrease in impurity")
#         plt.show()
#         ## END plotting feature importances
    
    
# ----- END OF ITERATIONS

print("\n =========== ALL ITERATIONS RESULTS SUMMARY ===========")
dfMetrics = pd.DataFrame({'iteration': [i for i in range(1,num_iters+1)], \
                          'mae': maes, 'mse': mses, 'rmse': rmses, 'corrs': corrs, 'ps': ps})
display(dfMetrics)

if shuffled:
    dfMetrics.to_csv(VALENCE_RESULTS + "RAW_SHUFFLED/" + model + "/" + model + "_Valence_Metrics_SHUFF.csv", index=False)
elif chance:
    dfMetrics.to_csv(VALENCE_RESULTS + "RAW_CHANCE/" + model + "/" + model + "_Valence_Metrics_CHANCE.csv", index=False)
else:
    dfMetrics.to_csv(VALENCE_RESULTS + "RAW/" + model + "/" + model + "_Valence_Metrics.csv", index=False)

print("Average over all iterations:")
print("%6s %.2f" % ("MAE:", np.mean(dfMetrics['mae'])))
print("%6s %.2f" % ("MSE:", np.mean(dfMetrics['mse'])))
print("%6s %.2f" % ("RMSE:", np.mean(dfMetrics['rmse'])))
print("%6s %.2f" % ("Corr:", np.mean(dfMetrics['corrs'])))
print("%6s %.7f" % ("p-val:", np.mean(dfMetrics['ps'])))



Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25

 =========== ALL ITERATIONS RESULTS SUMMARY ===========


,iteration,mae,mse,rmse,corrs,ps
0,1,0.567982,0.497321,0.705210,0.163164,0.007110
1,2,0.568326,0.495828,0.704151,0.181152,0.002761
2,3,0.566897,0.495012,0.703571,0.183132,0.002475
3,4,0.570229,0.500588,0.707522,0.146658,0.015683
4,5,0.572428,0.503443,0.709537,0.122625,0.043701
5,6,0.573306,0.504759,0.710464,0.092924,0.127019
6,7,0.573873,0.505575,0.711038,0.095828,0.115519
7,8,0.573109,0.505421,0.710929,0.090863,0.135709
8,9,0.567441,0.498867,0.706306,0.143354,0.018216
9,10,0.572523,0.501469,0.708145,0.126814,0.036943


Average over all iterations:
  MAE: 0.57
  MSE: 0.50
 RMSE: 0.71
 Corr: 0.13
p-val: 0.0624570


## without cross-validation

In [ ]:
features = ['REC', 'DET', 'ADL', 'MDL', 'DENTR', 'LAM', 'AVL', 'MVL', 'VENTR']

X = dfData.loc[:, features]  
y = dfData.loc[:, 'task_score']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

model_task_score = RandomForestRegressor(n_estimators=100, random_state=1, max_features='sqrt')


# Train model
model_task_score.fit(X_train, y_train)
    
# Predict    
y_pred = model_task_score.predict(X_test)



mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print("%5s %.2f" % ("MAE:", mae))
print("%5s %.2f" % ("MSE:", mse))
print("%5s %.2f" % ("RMSE:", rmse))






# Classification

## Predict Task Score 

### OHE labels

### Categorical

In [ ]:
features = ['REC', 'DET', 'ADL', 'MDL', 'DENTR', 'LAM', 'AVL', 'MVL', 'VENTR']

X = dfData.loc[:, features]  
y = dfData.loc[:, 'task_score_cat']

# y_bin = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

model_task_score = RandomForestClassifier(n_estimators=100, random_state=1, \
                                           max_features='sqrt', class_weight='balanced') 

# Train model
model_task_score.fit(X_train, y_train)
    
    
y_pred = model_task_score.predict(X_test)

# display(pd.DataFrame({"y_test": y_test, "y_pred": y_pred}))

print(classification_report(y_test, y_pred, zero_division=1))

cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True)


# #roc auc score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
# y_pp = model_task_score.predict_proba(X_test) 
# print("\ny_train info")
# print("total len: ", len(y_train))
# print("unique vals: ", len(set(y_train)))
# print("\ny_test info")
# print("total len: ", len(y_test))
# print("unique vals: ", len(set(y_test)))
# print("\ny_pp info")
# print("y_pp shape: ", y_pp.shape)

# auroc = roc_auc_score(y_test, y_pp, multi_class='ovo', average='weighted')
# print(auroc)



# Hyperparameter Tuning

## Random Search CV

### Classification

In [ ]:
features = ['REC', 'DET', 'ADL', 'MDL', 'DENTR', 'LAM', 'AVL', 'MVL', 'VENTR']

X = dfData.loc[:, features]  
y = dfData.loc[:, 'task_score_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
print("\n")


## Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

pprint(rf_random.best_params_)



base_model = RandomForestClassifier(n_estimators=100, random_state=1, max_features='sqrt') 
base_model.fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

print('Base Model Performance')
print(classification_report(y_test, base_predictions, zero_division=1))


best_random = rf_random.best_estimator_
random_predictions = best_random.predict(X_test)

print('Random Model Performance')
print(classification_report(y_test, random_predictions, zero_division=1))



### Regression

In [ ]:
# Resources
## https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# Split data
X_train, X_test, y_train, y_test = train_test_split(dfData.loc[:, features], dfData.loc[:, 'task_score'], \
                                                    test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test).reshape(-1, 1)

if model == "RFR":
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    pprint(random_grid)
    print("\n")
    
    ## Use the random grid to search for best hyperparameters
    
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X_train, y_train)

    pprint(rf_random.best_params_)

    base_model = RandomForestRegressor(n_estimators=100, random_state=1, max_features='sqrt') 
    base_model.fit(X_train, y_train)
    base_predictions = base_model.predict(X_test)
    base_mae = metrics.mean_absolute_error(y_test, base_predictions)

    print("===== Random Forest Regessor =====")
    print('Base Model Performance')
    print('MAE: ', np.round(base_mae, 2))


    best_random = rf_random.best_estimator_
    random_predictions = best_random.predict(X_test)
    random_mae = metrics.mean_absolute_error(y_test, random_predictions)

    print('Random Model Performance')
    print('MAE: ', np.round(random_mae, 2))


    print('Improvement of {:0.2f}%.'.format( 100 * (random_mae - base_mae) / base_mae))
    
elif model == "SVR":
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    
    degree = [1, 2, 3, 4, 5]
    
    gamma = ['scale', 'auto', 0.1]
    
    # Create the random grid
    random_grid = {'kernel': kernel,
                   'degree': degree,
                   'gamma': gamma}
    pprint(random_grid)
    print("\n")
    
    ## Use the random grid to search for best hyperparameters
    
    # First create the base model to tune
    sv = SVR()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    sv_random = RandomizedSearchCV(estimator = sv, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    sv_random.fit(X_train, y_train)

    pprint(sv_random.best_params_)

    base_model = SVR() 
    base_model.fit(X_train, y_train)
    base_predictions = base_model.predict(X_test)
    base_mae = metrics.mean_absolute_error(y_test, base_predictions)

    print("===== Support Vector Machine Regessor =====")
    print('Base Model Performance')
    print('MAE: ', np.round(base_mae, 2))


    best_random = sv_random.best_estimator_
    random_predictions = best_random.predict(X_test)
    random_mae = metrics.mean_absolute_error(y_test, random_predictions)

    print('Random Model Performance')
    print('MAE: ', np.round(random_mae, 2))


    print('Improvement of {:0.2f}%.'.format( 100 * (random_mae - base_mae) / base_mae))

